<center>
    <h1>Debugging and Profiling Python Code</h1>
    <h2>Andres Rios-Tascon</h2>
    <h4>Software Engineering Summer School 2024</h4>
    <img src="images/PU_lockup.png" style="height:50px;"/>
</center>

## Profiling and Debugging

One of the main tasks in software engineering is developing code that runs **fast** and **correctly**.

**Profiling** and **debugging** is how you achieve this.

- **Profiling:** The process of analyzing a program to measure its performance, usually in terms of execution time and memory usage, to identify bottlenecks and optimize efficiency.

- **Debugging:** The process of identifying, isolating, and fixing bugs or errors in a software program to ensure it runs correctly and efficiently.

Profiling and debugging differs depending on the language and platform you are working with, but the basics are the same.

Today we will be working with Python code, but the skills you will learn are transferable.

# Profiling Python Code

- Getting Python code to run efficiently can be tricky, especially since it is an interpreted language so it is inherently slower.

- Using built-in functions and compiled extension libraries, with proper code design, and with the aid of profiling tools, you can nevertheless write very efficient Python code.

- Today we'll be learning how to use `cProfile` and `line_profiler`.

## `cProfile`

- `cProfile` is a built-in profiler (along with `profile`, which is a pure-Python version).

- It provides similar functionality to what you would typically find in profilers for compiled languages (e.g. the Linux `perf` profiler).

- Gives us information about the functions that are running as we run our scripts, so that we can focus on improving the most time-consuming parts.

## `cProfile` example

Let's look at a simple example. The following code block contains a `fibonacci` method that computes the $n$-th element of the Fibonacci sequence with 3 different approaches.

$$F_{n}=F_{n-1}+F_{n-2}, \text{ with } F_{0}=0, F_{1}=1$$

In [ ]:
from functools import cache

def fib_recursive(n):
    if n <= 1:
        return n
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

@cache
def fib_recursive_cached(n):
    if n <= 1:
        return n
    else:
        return fib_recursive_cached(n-1) + fib_recursive_cached(n-2)

def fib_iterative(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

def fibonacci(n):
    f_re_n = fib_recursive(n)
    print(f"Fibonacci recursive ({n}): {f_re_n}")
    # Clear the cache so that profiling is accurate.
    # Don't use in real code.
    fib_recursive_cached.cache_clear()
    f_rec_n = fib_recursive_cached(n)
    print(f"Fibonacci recursive cached ({n}): {f_rec_n}")
    f_it_n = fib_iterative(n)
    print(f"Fibonacci iterative ({n}): {f_it_n}")

Let's try it out to see how it works.

In [ ]:
fibonacci(30)

Let's now see how to profile it. We first import `cProfile`.

In [ ]:
import cProfile

And then we can run the profiler like this.

In [ ]:
cProfile.run("fibonacci(30)")

You will typically have much longer functions, so it will be a lot harder to read through this output.

What we can do is output this information to a new file, and then use `pstats` to look through the data.

In [ ]:
cProfile.run("fibonacci(30)", "fibonacci_stats")

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats("fibonacci_stats")
p.strip_dirs().sort_stats(SortKey.FILENAME, SortKey.CALLS, SortKey.TIME).print_stats(23);

The idea now is to look for function calls that have large run time or number of calls. Once identified, you can start looking at the code to see what could be improved.

## `line_profiler`

- Since Python is an interpreted language, we can easily profile with more detail to see exactly which lines of code are taking a long time.

- This can be done with the `line_profiler` package, which can be installed with `pip install line_profiler`.

## `line_profiler` example

We can use this package in a Jupyter notebook by first loating it as follows.

In [ ]:
%load_ext line_profiler

Then, we use `%lprun` by specifying the functions that it should profile, as well as a statement to run. For example, for the Fibonacci example above, we would do it as follows.

In [ ]:
%lprun -f fibonacci fibonacci(30)

Since we see that most of the time is being spent in `fib_recursive`, let's also profile it.

In [ ]:
%lprun -f fibonacci -f fib_recursive fibonacci(30)

We can see that the recursion in `fib_recursive` is causing an excessive number of function calls, which is why it is so slow.

## Another example

Let's biefly look at another example. The following code block contains some functions to compute the sum of consecutive integers. Let's use the line profiler to look at their performance.

In [ ]:
def bad(n):
    total = 0
    for i in range(n):
        total += i
    return total

def good(n):
    return sum(i for i in range(n))

def best(n):
    return (n*(n-1))//2

def sum_of_integers(n):
    """
    Sum integers from 1 up to, but not including, n.
    Uses 3 different algorithms, to show performance difference.
    """
    res_bad = bad(n)
    print(f"Result from bad function ({n}): {res_bad}")
    res_good = good(n)
    print(f"Result from good function ({n}): {res_good}")
    res_best = best(n)
    print(f"Result from best function ({n}): {res_best}")

In [ ]:
%lprun -f bad -f good -f best -f sum_of_integers sum_of_integers(1000)

## Tips for improving performance

- Use built-in functions whenever possible.

- Find a library (ideally compiled) for the type of data/problem you are working with. E.g. if you want to work with numerical arrays, use `NumPy`.

- A good algorithm will always be the most important thing. However, this requires a good understanding of the computation.

## Exercise 1

Profile the following code and find ways to make it faster.

In [ ]:
import numpy as np

def calc_pi(num_trials):
    """Calculate pi using random numbers"""
    xarr = []
    yarr = []

    for _ in range(num_trials):
        xarr.append(np.random.rand())
        yarr.append(np.random.rand())

    in_circle = 0
    for x, y in zip(xarr, yarr):
        if x**2 + y**2 <= 1.0:
            in_circle += 1

    return 4*(in_circle/num_trials)

calc_pi(1_000_000)

In [ ]:
%lprun -f calc_pi calc_pi(1_000_000)

<details>
<summary>
Answer (Don't look until you're done)
</summary>
This would be a good way to make this function faster.
<pre><code>
def calc_pi(num_trials):
    """Calculate pi using random numbers"""
    x = np.random.rand(num_trials)
    y = np.random.rand(num_trials)

    in_circle = np.sum(x**2 + y**2 <= 1.0)

    return 4*(in_circle/num_trials)
</code></pre>

Note that we used functions built into NumPy instead of writing for-loops. Not only is the code faster, but shorter and more legible.
</details>

## Memory profiling

Unfortunately, memory profiling in Python is currently in a bit of a rough spot. `memory-profiler` and the `filprofiler` were good tools developed for this, but they are not actively supported. If you need to perform memory profiling, these might still be the best options. We won't cover these today.

# Debugging Python Code

- As you write code, you will inevitably make mistakes.

- Some bugs will be immediately obvious, but other will only show up in edge cases and will be difficult to tell what is wrong.

- Bugs that make the program crash are easier to debug in Python, since it tells you exactly where it failed. But logical bugs are equally difficult to solve.

- There are debugging tools that help you find and fix tricky bugs. They work similarly for other languages, so what you will learn here is transferable.

## The "classic" approach

- The first instinct for most people when encountering a bug is to add print statements in strategic places in the code to see what is happening.

- It is considered a "bad practice" by some people, but the truth is that all of us do it, and it is a good and quick approach to fix simple bugs.

- One thing that you should keep in mind is that prints are often buffered, so if the code crashes shortly after a print statement it might not end up printing anything. In Python you can force an unbuffered print with `print(..., flush=True)`.

## Using a debugger

- For complex bugs, it is much better to use a debugger, as it allows you to take a detailed look at what is happening as the program runs.

- Python comes with a built-in debugger called `pdb`, which works similarly to `gdb`.

## Debugging example

Let's consider the following broken function

In [ ]:
def my_broken_function():
    x = 6
    y = 4
    x += 2
    y *= 2
    x -= y
    y /= x
    return x, y

my_broken_function()

We can tell it to jump into the debugger by using the magic command `%debug` at the top of the cell. If you're running a python script, you can use `python -m pdb -c continue myscript.py`.

For a full description of how to use `pdb` you can visit [this page](https://docs.python.org/3/library/pdb.html). Let's try it out on the code above.

Often, it is better to start the debugger a bit before the crash happens, so we can step through each line and see what went wrong. We can do this with `import ipdb; ipdb.set_trace()`, or if you are not using Jupyter, simply with the `breakpoint()` function. Let's try it out above.

## Debugging example in JupyterLab

When working with JupyterLab, there is an alternative way to debug more directly and visually. You first need to enable debugging with the bug button at the top right. You can then use the debugging panel on the right.

Let's try it out with the following function.

In [ ]:
def average_of_5_largest(in_list):
    sorted_list = in_list.sort()
    largest_5 = sorted_list[-5:]
    sum_of_largest = sum(largest_5)
    length_of_largest = len(largest_5)
    average_of_largest = sum_of_largest/length_of_largest
    return average_of_largest

average_of_5_largest([0,1,2,3,4,5,6,7,8,9])
average_of_5_largest([0,1,2])
average_of_5_largest([])

## Exercise 2

Debug and fix the following code.

The `sylvester` function computes Sylvester's sequence up to the $n$-th element.
$$s_n=1+\prod_{i=0}^{n-1}s_i, \text{ with } s_0=2$$

The sequence should be $2, 3, 7, 43, 1807, 3263443, 10650056950807, \cdots$

In [ ]:
from math import prod

def add_to_list(*elements, starting_list=[]):
    starting_list.extend(elements)
    return starting_list

def sylvester(n):
    sequence = add_to_list(2)
    for _ in range(n-1):
        new_num = 1 + prod(sequence)
        sequence = add_to_list(new_num, starting_list=sequence)
    return sequence

print(sylvester(2))
print(sylvester(3))
print(sylvester(4))

<details>
<summary>
Answer (Don't look until you're done)
</summary>
By using breakpoints, we can find that sometimes at the the very start of the <code>add_to_list</code> function,
the <code>starting_list</code> is not empty even though we didn't specify it when calling the function.

This happens because default arguments for function are initialized when the function is defined. A mutable argument
will not be reinitialized for new function calls, so it keeps any changes that were made during previous calls.
This is a common mistake to make when writing Python code.

The correct function definition should be something like this
<pre><code>
def add_to_list(*elements, starting_list=None):
    if starting_list is None:
        starting_list = []
    starting_list.extend(elements)
    return starting_list
</code></pre>
<br/>
Note that this is a rather contrived example since we should be using <code>append</code> directly, but it shows a common mistake you might encounter.
</details>

# Summary

- As you write code, you need to ensure that it runs both **efficiently** and **correctly**. **Profiling** and **debugging** tools help you achieve this.

- Learning how to profile and debug properly takes practice. There's no step-by-step guide, so the best thing you can do is practice with your own projects.

- Today, I showed you some tools to profile and debug for Python, but the basics are the same for any other language.